In [29]:
# Import required libraries and project functions
import os
import requests
import openai
from dotenv import load_dotenv
from functions import fetch_movies_from_tmdb, generate_movie_recommendations, get_genres_for_mood

# Load environment variables from .env file
load_dotenv()

True

In [30]:
# TMDB API key is now loaded from Jupyter secrets (add with %store or JupyterLab secrets UI)
def ovies_from_tmdb(mood, decade, min_rating):
    """Fetch movies from TMDB based on mood, decade, minimum rating."""
    genres = get_genres_for_mood(mood)
    genre_str = ','.join(genres)
    start_date = f'{decade}-01-01'
    end_date = f'{decade+9}-12-31'
    api_key = os.environ.get('SECRET_TMDB_API_KEY')
    if not api_key:
        raise ValueError('TMDB API key not found. Please add TMDB_API_KEY as a Jupyter secret.')
    url = "https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "with_genres": genre_str,
        "sort_by": "popularity.desc",
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "vote_average.gte": min_rating,
    }
    response = requests.get(url, params=params)
    return response.json().get("results", [])

In [31]:
# Load OpenAI API key from .env file

openai_api_key = os.environ.get('OPENAI_API_KEY')

def generate_movie_recommendations(mood, movies, n=3):
    """Use OpenAI GPT to recommend and describe n movies for the given mood, showing TMDB rating next to each film name."""
    if not openai_api_key:
        raise ValueError('OpenAI API key not found. Please add OPENAI_API_KEY to your .env file.')
    openai.api_key = openai_api_key
    movie_list = '\n'.join([f"- {m['title']} (TMDB rating: {m.get('vote_average', 'N/A')}): {m.get('overview', 'No description available.')}" for m in movies[:10]])
    prompt = (f"User mood: {mood}\n"
              f"Here are some movies fetched from TMDB that might fit this mood (with their TMDB ratings):\n{movie_list}\n\n"
              f"Please recommend and describe {n} movies from this list that best fit the user's mood. For each recommended movie, you must display the TMDB rating next to the movie title in your output. Respond in a friendly, engaging way.")
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.7,
    )
    return response.choices[0].message.content

In [34]:
# Example usage: fetch movies and generate recommendations using the function
mood = 'happy'  # e.g., 'sad', 'excited', etc.
decade = 2000  # e.g., 1980, 1990, 2000, 2010, 2020
min_rating = 8.0  # Minimum average vote

movies = fetch_movies_from_tmdb(mood, decade, min_rating)

TypeError: fetch_movies_from_tmdb() missing 1 required positional argument: 'country'

In [ ]:
# Generate and display AI-powered recommendations
output = generate_movie_recommendations(mood, movies, n=10)
print(output)

2025-08-10 18:53:07.829 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I'm so glad to hear that you're feeling happy! Here are 10 movies that might just add to your cheerful mood:

1. Adventure Time (TMDB rating: 8.2) - Join Pen and Jake on a fun and adventurous journey to rescue Princess Bubblegum. This charming animated series is sure to bring a smile to your face.

2. Onni von Sopanen (TMDB rating: 9.0) - Follow 11-year-old Onni as he embarks on a heartwarming quest to discover his true roots. This touching story is bound to leave you feeling uplifted.

3. Lotte from Gadgetville (TMDB rating: 8.6) - Step into the whimsical world of Lotte and her inventive village as they compete in an annual competition. This delightful animated film is full of joy and creativity.

4. Codename: Kids Next Door: Operation I.N.T.E.R.V.I.E.W.S. (TMDB rating: 8.276) - Join the former KND operatives as they reminisce about their final mission. This action-packed and nostalgic adventure is sure to entertain you.

5. The 1st 13th Annual Fancy Anvil Awards Show Program Special: